In [1]:
import pandas as pd

In [2]:
ava_df = pd.read_csv('AVA.txt', sep=' ', names=['Image ID', 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 'Semantic ID 1', 'Semantic ID 2', 'Challenge ID'])
ava_df.head()

,Image ID,1,2,3,4,5,6,7,8,9,10,Semantic ID 1,Semantic ID 2,Challenge ID
1,953619,0,1,5,17,38,36,15,6,5,1,1,22,1396
2,953958,10,7,15,26,26,21,10,8,1,2,1,21,1396
3,954184,0,0,4,8,41,56,10,3,4,0,0,0,1396
4,954113,0,1,4,6,48,37,23,5,2,2,15,21,1396
5,953980,0,3,6,15,57,39,6,1,1,1,22,38,1396


In [3]:
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm

In [4]:
import numpy as np
rows_to_delete = []
for i, row in tqdm(ava_df.iterrows()):
    try:
        _ = Image.open('images/{}.jpg'.format(row['Image ID']))
    except:
        rows_to_delete.append(i)   

12611it [00:10, 1330.37it/s]/usr/local/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:754: UserWarning: Possibly corrupt EXIF data.  Expecting to read 37093376 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:754: UserWarning: Possibly corrupt EXIF data.  Expecting to read 20054016 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:771: UserWarning: Corrupt EXIF data.  Expecting to read 12 bytes but only got 10. 
  warnings.warn(str(msg))
26255it [00:21, 1094.64it/s]/usr/local/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:771: UserWarning: Corrupt EXIF data.  Expecting to read 12 bytes but only got 9. 
  warnings.warn(str(msg))
34347it [00:27, 1615.11it/s]/usr/local/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:754: UserWarning: Possibly corrupt EXIF data.  Expecting to read 84201912

99332it [01:21, 914.38it/s]/usr/local/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:754: UserWarning: Possibly corrupt EXIF data.  Expecting to read 4718592 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
112109it [01:30, 1397.13it/s]/usr/local/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:754: UserWarning: Possibly corrupt EXIF data.  Expecting to read 21652 bytes but only got 848. Skipping tag 700
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:754: UserWarning: Possibly corrupt EXIF data.  Expecting to read 4898 bytes but only got 848. Skipping tag 34377
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:754: UserWarning: Possibly corrupt EXIF data.  Expecting to read 940 bytes but only got 848. Skipping tag 34675
  " Skipping tag %s" % (size, len(data), tag))
115440it [01:33, 1350.67it/s]/usr/local/lib/python3.7/site-pa

241102it [03:06, 1411.92it/s]/usr/local/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:754: UserWarning: Possibly corrupt EXIF data.  Expecting to read 4832100392 bytes but only got 862. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:754: UserWarning: Possibly corrupt EXIF data.  Expecting to read 9664200784 bytes but only got 842. Skipping tag 512
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:754: UserWarning: Possibly corrupt EXIF data.  Expecting to read 2433155076 bytes but only got 947. Skipping tag 512
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:754: UserWarning: Possibly corrupt EXIF data.  Expecting to read 2433024001 bytes but only got 822. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:754: User

In [5]:
proc_ava_df = ava_df.drop(rows_to_delete)
print('Removed {} rows'.format(len(rows_to_delete)))

Removed 22 rows


In [6]:
proc_ava_df.to_csv('proc_AVA.csv', index=False)

In [7]:
ava_df = pd.read_csv('proc_AVA.csv')

In [8]:
ava_df.head()

,Image ID,1,2,3,4,5,6,7,8,9,10,Semantic ID 1,Semantic ID 2,Challenge ID
0,953417,0,0,0,5,32,50,23,10,3,1,22,0,1396
1,953777,0,3,2,3,13,40,35,21,8,3,20,53,1396
2,953756,0,2,3,9,35,50,20,5,2,2,0,0,1396
3,954195,0,1,7,26,56,23,6,1,0,2,0,0,1396
4,953903,0,1,4,5,33,50,17,9,3,2,21,28,1396


In [9]:
import numpy as np

def compute_mean_score(row):
    prob_distr = row[[1,2,3,4,5,6,7,8,9,10]].values
    prob_distr = prob_distr / prob_distr.sum()
    weights = np.arange(10) + 1
    mean_score = (prob_distr * weights).sum()
    return mean_score

In [10]:
%%time
ava_df['Mean Score'] = ava_df.apply(compute_mean_score, axis=1)

In [11]:
ava_df.head()

,Image ID,1,2,3,4,5,6,7,8,9,10,Semantic ID 1,Semantic ID 2,Challenge ID,Mean Score
0,953417,0,0,0,5,32,50,23,10,3,1,22,0,1396,6.112903
1,953777,0,3,2,3,13,40,35,21,8,3,20,53,1396,6.593750
2,953756,0,2,3,9,35,50,20,5,2,2,0,0,1396,5.796875
3,954195,0,1,7,26,56,23,6,1,0,2,0,0,1396,5.040984
4,953903,0,1,4,5,33,50,17,9,3,2,21,28,1396,5.943548


In [12]:
ava_df.to_csv('proc_AVA.csv', index=False)

In [13]:
len(ava_df)

255508